<a href="https://colab.research.google.com/github/dasnomp/Project-1-R7020E/blob/main/ObjectDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training code for YOLO v11 object detection
Worth noting is that all the paths need to be changed if you want to run the code, I didn't put any time on changing to relative paths

#Check if GPU is available



In [ ]:
!nvidia-smi

# Install Dependencies & Import Dataset from Roboflow

In [ ]:
!pip install ultralytics roboflow
import ultralytics
from roboflow import Roboflow
import os
ultralytics.checks()
rf = Roboflow(api_key="Ctgl3HHmuJuXANwCo7i9")
project = rf.workspace("fire-extinguisher").project("fireextinguisher-z5atr")
version = project.version(2)
dataset = version.download("yolov11")

# Import a pretrained YOLO 11 model and train on custom dataset

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")  # pretrained model

model.train(
    data="FireExtinguisher-2/data.yaml",
    epochs=500, # set epochs to large number to able to use resume=True later
    imgsz=640,
    resume = False
)

# Save training results in Google Drive



In [ ]:
from google.colab import drive
import os, shutil

# 1. Mount drive
drive.mount('/content/drive')

# 2. Base folder in drive
base_dir = "/content/drive/MyDrive/Colab Notebooks/yolo_sessions"
os.makedirs(base_dir, exist_ok=True)

# 3. Find next session number to save in
# This code snippet creates folder according to "Session_n" where n = 1,2,3,...
existing = [d for d in os.listdir(base_dir) if d.startswith("Session_")]
nums = [int(d.split("_")[1]) for d in existing if d.split("_")[1].isdigit()]
next_num = max(nums, default=0) + 1
session_dir = os.path.join(base_dir, f"Session_{next_num}")
os.makedirs(session_dir, exist_ok=True)

# 4. Save target data
src_dir = "runs/detect/train"
targets = [
    "weights/best.pt",
    "weights/last.pt",
    "results.csv", # Save csv files to get data from start to end
    "args.yaml"
]

# 5. Copy
for rel_path in targets:
    full_src = os.path.join(src_dir, rel_path)
    if os.path.exists(full_src):
        shutil.copy(full_src, session_dir)
        print(f"✅ Saved {rel_path}")
    else:
        print(f"⚠️ Couldn't find {rel_path} — Skipping.")

print(f"\n📂 Saved in: {session_dir}")

# Resume training with previous weights imported from Google Drive

In [ ]:
from google.colab import drive
from ultralytics import YOLO
drive.mount('/content/drive/')
model = YOLO('/content/drive/MyDrive/Colab Notebooks/yolo_sessions/Session_3/last.pt')

In [ ]:
model.train(
    data="FireExtinguisher-2/data.yaml",
    epochs=500,
    imgsz=640,
    resume=True
)

#Additional 30 epochs of training with Augmented Data

In [ ]:
from ultralytics import YOLO

model = YOLO('runs/detect/train_aug_test/weights/last.pt')

model.train(
    data="FireExtinguisher-2/data.yaml",
    project="runs/detect",
    name="train_aug_test",
    epochs=30,
    imgsz=640,
    lr0=0.005,               # small learning rate to finetune
    # augment the data:
    perspective=0.003, shear=0.2, scale=0.6, hsv_v=0.6,
    translate=0.15, copy_paste=0.2, erasing=0.4,
)